

#  富信息图神经网络（Rich-Information GNNs）
![](./img/4_1.png)

现实中的图往往**包含更多结构信息或属性信息**，比如：

* 有向关系（如引用网络）
* 多类型节点与边（如知识图谱）
* 时变动态（如社交网络、交易网络）

这些复杂特性超出了标准 GCN、GAT 等“静态、无向、同质图”的处理能力。
因此，研究者发展出**富信息图神经网络**来增强 GNN 对这些图的建模能力。

---

## 1 有向图神经网络（Directed Graph Neural Networks）

---

###  有向图的挑战

标准 GCN/GAT 假设图是**无向图**，邻居特征聚合默认“对称”：

* 现实中许多图是**有向图**（方向携带语义）：

  * 引用图（被引用 → 影响力）
  * 知识图谱（subject → predicate → object）
  * 推荐系统（用户 → 点击 → 商品）

---

###  建模方向性的方法

####  1.1 邻接矩阵改造法

* 保留方向性邻接矩阵 $A_{dir}$，不对称聚合：

  * 区分**入边**邻居和**出边**邻居
  * 分别定义 $\text{IN}$ 和 $\text{OUT}$ 聚合器

####  1.2 双向信息传播（Bi-Propagation）

$$
h_v^{(k)} = \text{AGG}_{\text{in}}(\{h_u: u \rightarrow v\}) + \text{AGG}_{\text{out}}(\{h_u: v \rightarrow u\})
$$

* 将**入边和出边**视作两类不同语义通道，分别建模再融合。

####  1.3 方向编码（Positional Encoding）

* 给每条边添加“方向嵌入”作为额外特征；
* 在消息传递中用 $\text{MLP(edge direction)}$ 引入方向信息。

####  1.4 代表模型：DGCN、DiGCN、PDGNN 等









## 2 异构图神经网络（Heterogeneous Graph Neural Networks, HGNN）

---

###  异构图定义

> 图中包含**多类型节点和/或多类型边**的结构，称为**异构图**。

形式定义：

$$
G = (V, E, \phi_V, \phi_E)
$$

* $\phi_V(v) \in \mathcal{T}_V$：节点类型
* $\phi_E(e) \in \mathcal{T}_E$：边类型（语义）

---

###  异构图的挑战

| 问题    | 描述                 |
| ----- | ------------------ |
| 多类型节点 | 表示空间不一致            |
| 多种关系  | 边语义复杂，不可统一处理       |
| 多尺度语义 | 同一节点类型通过不同路径具有不同含义 |

---

###  关键建模策略

####  2.1 基于元路径（Meta-Path）方法

* 定义语义路径，如：作者-论文-会议-论文-作者（A-P-C-P-A）
* 使用路径引导邻接关系构建子图，聚合语义子图中的节点表示

代表模型：**HAN（Heterogeneous Attention Network）**

$$
h_v = \sum_{\text{meta-path } m} \alpha_m \cdot \text{GNN}_m(v)
$$

* 对不同元路径构建子图
* 引入路径注意力进行语义融合

---

####  2.2 基于元图（Meta-Graph）方法

* 使用更复杂结构（如图模式）定义语义依赖
* 更高表达能力但计算更重

---

####  2.3 关系级聚合器（Relation-specific Aggregation）

* 为每种边类型设计单独的变换矩阵：

$$
h_v = \sigma\left( \sum_{r \in \mathcal{R}} \sum_{u \in \mathcal{N}_r(v)} \frac{1}{c_{vr}} W_r h_u \right)
$$

代表模型：**R-GCN（Relational-GCN）**、**HGT（Heterogeneous GAT）**

---

####  2.4 类型对齐投影（Type-specific Projection）

* 将不同类型节点/边映射到同一语义空间，再做统一消息传递：

$$
h_v = \text{MLP}_{\phi(v)}(x_v)
$$



## 3 动态图神经网络（Dynamic Graph Neural Networks, DGNN）

---

###  动态图定义

动态图是随时间变化的图结构，包含：

* 节点变化（加入/移除）
* 边的变化（关系建立或消失）
* 节点/边属性随时间变化（如交易金额）

---

###  建模分类

####  3.1 时间切片图（Snapshot-based GNN）

* 将图按照时间分割为一系列静态快照 $G^{(t)}$
* 在每一帧上用静态 GNN 提取表示，再用 RNN/Transformer 建模时间序列

代表模型：

* **EvolveGCN**：将 GCN 权重随时间演化
* **GC-LSTM**：图卷积结合 LSTM 处理图序列

---

####  3.2 事件驱动图（Event-based DGNN）

* 以事件（如交易、点赞）为输入，图结构实时更新
* 对每个事件触发节点更新，用时间编码建模影响

代表模型：

* **TGAT（Temporal Graph Attention）**：边上编码时间信息
* **TGN（Temporal Graph Network）**：节点状态随着时间递推更新

---

####  3.3 持续时间建模（Temporal Point Process）

* 基于 Hawkes Process 或时序概率建模事件发生概率
* 可预测未来事件和发生时间

---

###  核心挑战

| 挑战   | 说明              |
| ---- | --------------- |
| 表达能力 | 如何同时捕捉结构与时间演化   |
| 计算效率 | 实时更新 + 历史保持难度大  |
| 可预测性 | 如何预测未来结构/属性演化趋势 |

---

##  小结

| 模型类型    | 适用图         | 关键挑战      | 代表方法                 |
| ------- | ----------- | --------- | -------------------- |
| 有向图 GNN | 单一类型节点，有方向边 | 方向性建模     | DGCN, DiGCN          |
| 异构图 GNN | 多类型节点/边     | 类型表达、语义聚合 | HAN, R-GCN, HGT      |
| 动态图 GNN | 随时间变化的图     | 演化建模、时序预测 | TGAT, TGN, EvolveGCN |